### KPI - Availability (Sev1: Burn Velocity > 14x in 5 mins windows)

In [1]:
! pip install azure-identity azure-monitor-query langchain-openai


In [2]:
## Get the credential to query log analytics
from azure.identity import AzureCliCredential
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
import pandas as pd

# Azure config
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
workspace_id = "edf08e1f-916f-48c3-bc52-273492d63c8f" #data connector
# workspace_id = "edf08e1f-916f-48c3-bc52-273492d63c8f" #msonecloud tool
#workspace_id = "5938c293-3317-4e63-9b33-a248eb20cc81" #blogging sites

# Auth
credential = AzureCliCredential()
client = LogsQueryClient(credential)

In [3]:
# Calculate the Burn Rate for slo = 99.95% in 5 minutes window
# Fire this alert in every 5 minutes

slo = 0.9995
timeGrain = "5m"

query = f"""

let SLO = {slo};
let timeGrain = {timeGrain};
AzureDiagnostics
| where TimeGenerated > ago(24d)
  and Resource == "MSONECLOUDAPIFD"
  and Category == "FrontDoorAccessLog"
| where isnotempty(httpStatusCode_s)
| extend statusCode = toint(httpStatusCode_s)
| summarize
    totalRequests = count(),
    errorRequests = countif(statusCode >= 500)
    by bin(TimeGenerated, timeGrain)
| extend SLI = iff(totalRequests > 0, 1.0 - (1.0 * errorRequests / totalRequests), 1.0)
| extend burnRate = iff(totalRequests > 0, (1.0 - SLI) / (1.0 - SLO), 0.0)
| project TimeGenerated, totalRequests, errorRequests, SLI, burnRate
| order by TimeGenerated asc
"""

# Query execution
response = client.query_workspace(
    workspace_id=workspace_id,
    query=query,
    timespan=None
)

# Parse results
if response.status == LogsQueryStatus.SUCCESS:
    table = response.tables[0]
    df = pd.DataFrame(data=table.rows, columns=table.columns)
    df["TimeGenerated"] = pd.to_datetime(df["TimeGenerated"])
    df = df.sort_values("TimeGenerated")

max_burnrate_df = df[(df["burnRate"] == df["burnRate"].max()) & (df["burnRate"].max() >= 1) ]
# max_burnrate_df = df[df["burnRate"] == df["burnRate"].max() ]
# max_burnrate_df = df[df["burnRate"] >= 14]
print(max_burnrate_df)

if max_burnrate_df.shape[0] > 0:
    max_burn_row = max_burnrate_df.iloc[0]
    print(max_burn_row)

print("Max burn rate:", type(max_burn_row["TimeGenerated"]))


                TimeGenerated  totalRequests  errorRequests       SLI  \
869 2025-06-24 23:05:00+00:00          10269           1388  0.864836   

       burnRate  
869  270.328172  
TimeGenerated    2025-06-24 23:05:00+00:00
totalRequests                        10269
errorRequests                         1388
SLI                               0.864836
burnRate                        270.328172
Name: 869, dtype: object
Max burn rate: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [6]:
import requests

if max_burnrate_df.shape[0] > 0:
    max_burn_row = max_burnrate_df.iloc[0]
    # API_URL = "http://127.0.0.1:8000/analyze-burnrate"
    API_URL = "https://ai-alert-service.azurewebsites.net/analyze-burnrate"
    incidentTime = max_burn_row["TimeGenerated"]
    burnRate = max_burn_row["burnRate"]

    # Convert to ISO 8601 format if needed
    incidentTime_str = str(incidentTime)

    # print(incidentTime_str)

    payload = {
        "appName": "msonecloudapi",  # Change as needed
        "burnRate":str( burnRate),
        "incidentTime": incidentTime.isoformat(),  # ISO format string
        "severity": "sev2",
        "bNotify": True
    }
    print("Payload for LLM:", payload)
    print(incidentTime.isoformat()) 

    response = requests.post(API_URL, json=payload, timeout=120)

    if response.ok:
        result = response.json().get("result", "")
        print("Incident Analysis from LLM:\n")
        print(result)
    else:
        print("Error:", response.status_code, response.text)

Payload for LLM: {'appName': 'msonecloudapi', 'burnRate': '270.3281721686928', 'incidentTime': '2025-06-24T23:05:00+00:00', 'severity': 'sev2', 'bNotify': True}
2025-06-24T23:05:00+00:00


ReadTimeout: HTTPSConnectionPool(host='ai-alert-service.azurewebsites.net', port=443): Read timed out. (read timeout=120)